# Part I: Data Collection

## Task 1

Create a function that takes the file we just downloaded and returns a CSV file with the following columns:

In [1]:
# Importing the required modules
import os
import pandas as pd
from bs4 import BeautifulSoup

def read_file(File_name):

    share = '/work/' + File_name
    # empty list
    data = []

    # for getting the header from
    # the HTML file
    header = []
    html = BeautifulSoup(open(share),'html.parser')
    head = html.find_all("table")[0].find("tr")# find all tables in the fille and return the first , than find the 'tr' in this table

    for items in head:
        try:
            header.append(items.get_text())
        except:
            continue

    # let's get the data
    HTML_data = html.find_all("table")[0].find_all("tr")[1:]
    # put the data getted in the varible data 
    for element in HTML_data:
        list_data = []
        for list_element in element:
            try:
                list_data.append(list_element.get_text())
            except:
                continue
        data.append(list_data)

    
    # transform to DataFrame 
    dataF = pd.DataFrame(data = data, columns = header)
    dataF.drop(dataF.columns[[0,7]], axis=1, inplace=True)  # Delete the culumns that are not needed 
    dataF.columns=['date','closing','adjusted','evolution','quantity','volume'] # Rename the columns
    dataF=dataF.apply(lambda x: x.str.replace(',','.')) # replace ',' with '.' in order to read successfully the data 
    dataF=dataF.apply(lambda x: x.str.replace('\n',''))  # replace '\' with ' ' in order to read successfully the data 
    dataF['closing']=dataF['closing'].astype('float')  # reverse types 
    dataF['adjusted']=dataF['adjusted'].astype('float')  # reverse types 
    dataF['evolution']=dataF['evolution'].astype('float')  # reverse types 
    dataF['quantity']=dataF['quantity'].astype('float')  # reverse types 
    dataF['volume']=dataF['volume'].astype('float') # reverse types 
    dataF['date']= pd.to_datetime(dataF['date'])  # reverse types 
    dataF=dataF.sort_values('date')
    dataF=dataF.reset_index()
    dataF=dataF.drop('index',axis=1)
    
    return dataF


application of the declared function

In [2]:
AFM =read_file('AFM AS .aspx')
AFM

,date,closing,adjusted,evolution,quantity,volume
0,2020-01-04,979.0,979.0,-0.31,1.0,979.0
1,2020-01-06,1004.0,1004.0,0.40,1.0,1004.0
2,2020-01-07,986.0,986.0,-1.89,8.0,7888.0
3,2020-01-09,1005.0,1005.0,0.00,1.0,1005.0
4,2020-01-10,1005.0,1005.0,0.00,1.0,1005.0
...,...,...,...,...,...,...
494,2021-12-27,1319.0,1319.0,0.00,1.0,1319.0
495,2021-12-28,1344.0,1344.0,1.90,19.0,25536.0
496,2021-12-29,1370.0,1370.0,1.93,20.0,27400.0
497,2021-12-30,1290.0,1290.0,-5.84,104.0,142044.0


In [3]:
AFM.to_csv('AFM.csv')

Let's do the same thing with the other shares: AGM, ATL, SAN, WF.

In [4]:
# AGMA Assurance
AGM =read_file('AGM AS .aspx')
AGM.to_csv('AGM.csv')

# ATLANTA SANAD Assurance
ATL =read_file('ATL ASS .aspx')
ATL.to_csv('ATL.csv')

# SANLAM Assurace 
SAN =read_file('SAN ASS .aspx')
SAN.to_csv('SAN.csv')

# WAFA Assurance 
WF =read_file('WF ASS .aspx')
WF.to_csv('WF.csv')

# Part II : Data Processing 

Now that we have collected the necessary data, it is time to make use of it.

## Task 2

Let's create a class Stock that will represent stock in this project. This class should have at least the following instance attributes: 

- name: The name of the stock it represents.

- data: A Pandas Dataframe generated from the CSV of the previous section.

In [5]:
#the Class Stock
class Stock:

    def __init__(self, name):
        self.name = name
        self.data = read_file(name+' .aspx')

S1 = Stock('AFM AS')

print('Name:\t',S1.name,'\n\n\n')
print('Data :\n',S1.data)


Name:	 AFM AS 



Data :
           date  closing  adjusted  evolution  quantity    volume
0   2020-01-04    979.0     979.0      -0.31       1.0     979.0
1   2020-01-06   1004.0    1004.0       0.40       1.0    1004.0
2   2020-01-07    986.0     986.0      -1.89       8.0    7888.0
3   2020-01-09   1005.0    1005.0       0.00       1.0    1005.0
4   2020-01-10   1005.0    1005.0       0.00       1.0    1005.0
..         ...      ...       ...        ...       ...       ...
494 2021-12-27   1319.0    1319.0       0.00       1.0    1319.0
495 2021-12-28   1344.0    1344.0       1.90      19.0   25536.0
496 2021-12-29   1370.0    1370.0       1.93      20.0   27400.0
497 2021-12-30   1290.0    1290.0      -5.84     104.0  142044.0
498 2021-12-31   1360.0    1360.0       5.43      20.0   26675.0

[499 rows x 6 columns]


## Task 3

Inside Stock, create a method responsible for visualization. It should be able to create something similar to the graph present on the website.

In [6]:
DeepnoteChart(AFM, """{"mark":{"clip":true,"type":"trail","color":"#4c78a8","tooltip":true},"config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{"x":{"sort":"ascending","type":"temporal","field":"date","scale":{"type":"linear","zero":false}},"y":{"sort":null,"type":"quantitative","field":"closing","scale":{"type":"linear","zero":false},"aggregate":"sum"}}}""")

In [7]:
from plotly.subplots import make_subplots # in order to make sub plots, different plots in the same garphique
import plotly.graph_objects as go # type of graphiques 
def visualization(df, title):
    z = pd.DataFrame(df.data)
    df = z [['date','closing','volume']]
   
    df['date'] = pd.to_datetime(df['date'], format='%y%m%d') 

    # set the date as index
    df1 = df.set_index('date') 

    # Add an ather axis to my plot
    fig2 = make_subplots(specs=[[{"secondary_y": True}]]) 

    # plot the closing price 
    fig2.add_trace(go.Scatter(x=df1.index,y=df1['closing'],name='Price'),secondary_y=False,) 

    # plot the volume 
    fig2.add_trace(go.Bar(x=df1.index,y=df1['volume'],name='volume'),secondary_y=True,)

    # update the y axes
    #fig2.update_yaxes(visible=True, secondary_y=True) 
    fig2.update_yaxes(visible=True, secondary_y=False)

    # show the plots 
    fig2.update_layout(title={'text': title, 'x': 0.4})
    fig2.show()







### Application of the function for AFMA Assurance conseil

In [8]:
AF = Stock('AFM AS')
visualization(AF, 'AFMA Assurance conseil')

/tmp/ipykernel_157/1936231521.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'], format='%y%m%d')


### Application of the function for AGMA Assurance conseil

In [9]:
AG = Stock('AGM AS')
visualization(AG, 'AGMA Assurance ')

/tmp/ipykernel_157/1936231521.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Application of the function for ATLANTA SANAD

In [10]:
ATL = Stock('ATL ASS')
visualization(ATL, 'ATLANTA SANAD ')

/tmp/ipykernel_157/1936231521.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Application of the function for SANLAM ASSURANCE

In [11]:
SAN = Stock('ATL ASS')
visualization(SAN, 'ATLANTA SANAD ')

/tmp/ipykernel_157/1936231521.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Application of the function for WAFA ASSURANCE

In [12]:
ATL = Stock('ATL ASS')
visualization(ATL, 'ATLANTA SANAD ')

/tmp/ipykernel_157/1936231521.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# 

## Task 4:

Inside Stock, let's create the following instance methods:


- a method that returns the maximum value in the Dataframe.

- a method that returns the minimum value in the Dataframe.

- a method that returns the maximum volume in the Dataframe.

- a method that returns the maximum quantity in the Dataframe.

### understand the algorithm : 

In [13]:
SAN = Stock('ATL ASS')
SAN.data

,date,closing,adjusted,evolution,quantity,volume
0,2020-01-04,61.00,61.00,-2.24,452.0,27647.00
1,2020-01-06,65.39,65.39,0.00,0.0,0.00
2,2020-01-07,68.84,68.84,-1.45,2227.0,149765.36
3,2020-01-09,73.00,73.00,0.69,2231.0,162263.05
4,2020-01-10,78.50,78.50,0.00,1389.0,108405.60
...,...,...,...,...,...,...
494,2021-12-27,117.60,117.60,0.51,670.0,78067.85
495,2021-12-28,117.00,117.00,-0.51,1301.0,151858.00
496,2021-12-29,114.65,114.65,-2.01,568.0,66262.00
497,2021-12-30,120.00,120.00,4.67,38994.0,4669827.85


1. First method

Let's define the methods, and insert it in the class Stock

In [14]:
# method that returns the maximum value in the Dataframe.
def df_maxp(self):
    return max(self.data['closing'])

# a method that returns the minimum value in the Dataframe.
def df_minp(self):
    return min(self.data['closing'])

# a method that returns the maximum volume in the Dataframe.
def df_maxv(self):
    return max(self.data['volume'])

# a method that returns the maximum quantity in the Dataframe.
def df_maxq(self):
    return max(self.data['quantity'])



In [15]:
class Stock1:

    def __init__(self, name):
        self.name = name
        self.data = read_file(name+' .aspx')
        z = pd.DataFrame(self.data)
        self.df = z


    
Stock1.df_maxp = df_maxp
Stock1.df_minp = df_minp
Stock1.df_maxv = df_maxv
Stock1.df_maxq = df_maxq


### Application of the function for ATLANTA Assurance

In [16]:
ATL = Stock1('ATL ASS')
print(ATL.df_maxp() )
print(ATL.df_minp() )
print(ATL.df_maxv() )
print(ATL.df_maxq() )

122.0
56.0
18135803.4
200001.0


2. Second method

In [17]:
class Stock2:

    def __init__(self, name):
        self.name = name
        self.data = read_file(name+' .aspx')
        z = pd.DataFrame(self.data)
        self.df = z

    # method that returns the maximum value in the Dataframe.
    def df_maxp(self):
        return max(self.data['closing'])
    # a method that returns the minimum value in the Dataframe.
    def df_minp(self):
        return min(self.data['closing'])
                
    # a method that returns the maximum volume in the Dataframe.
    def df_maxv(self):
        return max(self.data['volume'])
            
    # a method that returns the maximum quantity in the Dataframe.
    def df_maxq(self):
        return max(self.data['quantity'])
    

In [18]:
ATL = Stock2('ATL ASS')
print(ATL.df_maxp() )
print(ATL.df_minp() )
print(ATL.df_maxv() )
print(ATL.df_maxq() )

122.0
56.0
18135803.4
200001.0


In [19]:
print(ATL.df_maxp() )

122.0


## Task 5 :

First method

In [20]:
class Stock:

    def __init__(self, name):
        self.name = name
        self.data = read_file(name+' .aspx')
       # z = pd.DataFrame(self.data)
       # self.df = z
        self.df = pd.DataFrame(self.data)

    # method that returns the maximum value in the Dataframe.
    def df_maxp(self):
        return max(self.data['closing'])

    # a method that returns the minimum value in the Dataframe.
    def df_minp(self):
        return min(self.data['closing'])
                
    # a method that returns the maximum volume in the Dataframe.
    def df_maxv(self):
        return max(self.data['volume'])
            
    # a method that returns the maximum quantity in the Dataframe.
    def df_maxq(self):
        return max(self.data['quantity'])

    #  create a method responsible for visualization. 
    def visualization(self, df, name):
        visualization(df, name)


### Application of the function for ATLANTA Assurance

In [21]:
ATL = Stock('ATL ASS')
ATL.visualization(ATL,'ATL ASS')

/tmp/ipykernel_157/1936231521.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Second method

In [22]:
class Stock:

    def __init__(self, name):
        self.name = name
        self.data = read_file(name+' .aspx')
       # z = pd.DataFrame(self.data)
       # self.df = z
        self.df = pd.DataFrame(self.data)

    # method that returns the maximum value in the Dataframe.
    def df_maxp(self):
        return max(self.data['closing'])

    # a method that returns the minimum value in the Dataframe.
    def df_minp(self):
        return min(self.data['closing'])
                
    # a method that returns the maximum volume in the Dataframe.
    def df_maxv(self):
        return max(self.data['volume'])
            
    # a method that returns the maximum quantity in the Dataframe.
    def df_maxq(self):
        return max(self.data['quantity'])

    #  create a method responsible for visualization. 
    def visualization(self,df, title):
        z = pd.DataFrame(df.data)
        df = z [['date','closing','volume']]
   
        df['date'] = pd.to_datetime(df['date'], format='%y%m%d') # 

        # set the date as index
        df1 = df.set_index('date') 

        # Add an ather axis to my plot
        fig2 = make_subplots(specs=[[{"secondary_y": True}]]) 

        # plot the closing price 
        fig2.add_trace(go.Scatter(x=df1.index,y=df1['closing'],name='Price'),secondary_y=False,) 

        # plot the volume 
        fig2.add_trace(go.Bar(x=df1.index,y=df1['volume'],name='volume'),secondary_y=True,)

        # update the y axes
        fig2.update_yaxes(visible=True, secondary_y=True) 
        fig2.update_yaxes(visible=True, secondary_y=False)

        # show the plots 
        fig2.update_layout(title={'text': title, 'x': 0.4})
        fig2.show()

    

In [23]:
ATL = Stock('ATL ASS')
ATL.visualization(ATL,'ATL ASS')

/tmp/ipykernel_157/3791555288.py:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Task 6:

Inside Stock, let's create a method that calculates the momentum for the given stock for any value of N. Your method should also accept a list of values for N; ie, it should be able to compute the momentum for various values of N at once. This behavior is tantamount to calling the method for each N in the list.

### Understand the algorithm :

In [31]:
# let's define a functuon that calculates the momentum:
ATL = Stock('ATL ASS')
 
# method that calculates the momentum
def moment( df, N): # df type of stock (class), and N list
    L = []
    n = len(df.data)
    clos = df.data['closing']
    L =[ [clos[n-j]-clos[n-i-j] for j in range(1, n-i+1)] for i in N] 
    return L

N = [5, 20, 120, 252]
# 5 weekly momentum
# 20 monthly momentum
# 120 semetsterly momentum
# 252 yearly momentum

moment(ATL, N)



[[5.0,
  2.200000000000003,
  -0.3499999999999943,
  2.75,
  2.5999999999999943,
  -1.0,
  1.7999999999999972,
  -1.0,
  -1.8499999999999943,
  -1.0,
  3.5,
  0.0,
  23.0,
  23.129999999999995,
  22.5,
  23.5,
  36.599999999999994,
  12.0,
  10.969999999999999,
  -20.5,
  -23.0,
  -36.599999999999994,
  -35.900000000000006,
  -34.95,
  -0.7000000000000028,
  -3.9000000000000057,
  -1.7000000000000028,
  1.9000000000000057,
  0.45000000000000284,
  -1.7000000000000028,
  2.4000000000000057,
  4.6000000000000085,
  21.53,
  22.049999999999997,
  23.400000000000006,
  36.400000000000006,
  33.08999999999999,
  -25.430000000000007,
  -24.049999999999997,
  -28.849999999999994,
  -39.900000000000006,
  -36.989999999999995,
  2.4000000000000057,
  1.7000000000000028,
  4.099999999999994,
  1.0,
  -0.7000000000000028,
  1.5,
  0.7999999999999972,
  1.7999999999999972,
  18.5,
  24.700000000000003,
  21.010000000000005,
  36.900000000000006,
  34.95,
  -2.0,
  -8.5,
  -7.510000000000005,
  -22

let's insert the function inside of the class stock 

In [26]:
class Stock:

    def __init__(self, name):
        self.name = name
        self.data = read_file(name+' .aspx')
       # z = pd.DataFrame(self.data)
       # self.df = z
        self.df = pd.DataFrame(self.data)

    # method that returns the maximum value in the Dataframe.
    def df_maxp(self):
        return max(self.data['closing'])

    # a method that returns the minimum value in the Dataframe.
    def df_minp(self):
        return min(self.data['closing'])
                
    # a method that returns the maximum volume in the Dataframe.
    def df_maxv(self):
        return max(self.data['volume'])
            
    # a method that returns the maximum quantity in the Dataframe.
    def df_maxq(self):
        return max(self.data['quantity'])

    #  create a method responsible for visualization. 
    def visualization(self,df, title):
        z = pd.DataFrame(df.data)
        df = z [['date','closing','volume']]
   
        df['date'] = pd.to_datetime(df['date'], format='%y%m%d') # 

        # set the date as index
        df1 = df.set_index('date') 

        # Add an ather axis to my plot
        fig2 = make_subplots(specs=[[{"secondary_y": True}]]) 

        # plot the closing price 
        fig2.add_trace(go.Scatter(x=df1.index,y=df1['closing'],name='Price'),secondary_y=False,) 

        # plot the volume 
        fig2.add_trace(go.Bar(x=df1.index,y=df1['volume'],name='volume'),secondary_y=True,)

        # update the y axes
        fig2.update_yaxes(visible=True, secondary_y=True) 
        fig2.update_yaxes(visible=True, secondary_y=False)

        # show the plots 
        fig2.update_layout(title={'text': title, 'x': 0.4})
        fig2.show()
    
    # method that calculates the momentum
    def moment(self, df, N):
        L = []
        n = len(df.data)
        clos = df.data['closing']
        L =[ [clos[n-j]-clos[n-i-j] for j in range(1, n-i+1)] for i in N] 
        return L
    

    

### Application of the function for ATLANTA Assurance

In [27]:
ATL = Stock('ATL ASS')

N = [5, 20, 120, 252]
print(len(ATL.data))
T = ATL.moment(ATL, N)
print(len(T[1]))# verify the length of the new data 

499
479


## Task 7:

Inside Stock, let's create a method that calculates the simple moving average for the given stock.

### Understand the algorithm:

In [32]:
ATL = Stock('ATL ASS')
k = 20
L = pd.DataFrame([])
clos = ATL.data['closing']
n = len(ATL.data)
L = clos.rolling(k).mean()
L

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
494    106.1060
495    107.8560
496    109.5385
497    111.5685
498    113.1185
Name: closing, Length: 499, dtype: float64

### Understand the algorithm:

In [33]:
# funtion that calculates the simple movign average :
def SMA(df, k):
    L = pd.DataFrame([])
    clos = df.data['closing']
    n = len(df.data)
    L = clos.rolling(k).mean()
    return L
    



In [34]:
ATL = Stock('ATL ASS')
k = 20
SMA(ATL, k)


0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
494    106.1060
495    107.8560
496    109.5385
497    111.5685
498    113.1185
Name: closing, Length: 499, dtype: float64

### Insert the function inside the class stock 

In [43]:
class Stock:

    def __init__(self, name):
        self.name = name
        self.data = read_file(name+' .aspx')
       # z = pd.DataFrame(self.data)
       # self.df = z
        self.df = pd.DataFrame(self.data)

    # method that returns the maximum value in the Dataframe.
    def df_maxp(self):
        return max(self.data['closing'])

    # a method that returns the minimum value in the Dataframe.
    def df_minp(self):
        return min(self.data['closing'])
                
    # a method that returns the maximum volume in the Dataframe.
    def df_maxv(self):
        return max(self.data['volume'])
            
    # a method that returns the maximum quantity in the Dataframe.
    def df_maxq(self):
        return max(self.data['quantity'])

    #  create a method responsible for visualization. 
    def visualization(self,df, title):
        z = pd.DataFrame(df.data)
        df = z [['date','closing','volume']]
   
        df['date'] = pd.to_datetime(df['date'], format='%y%m%d') 

        # set the date as index
        df1 = df.set_index('date') 

        # Add an ather axis to my plot
        fig2 = make_subplots(specs=[[{"secondary_y": True}]]) 

        # plot the closing price 
        fig2.add_trace(go.Scatter(x=df1.index,y=df1['closing'],name='Price'),secondary_y=False,) 

        # plot the volume 
        fig2.add_trace(go.Bar(x=df1.index,y=df1['volume'],name='volume'),secondary_y=True,)

        # update the y axes
        fig2.update_yaxes(visible=True, secondary_y=True) 
        fig2.update_yaxes(visible=True, secondary_y=False)

        # show the plots 
        fig2.update_layout(title={'text': title, 'x': 0.4})
        fig2.show()
    
    # method that calculates the momentum
    def moment(self, df, N):
        L = []
        n = len(df.data)
        clos = df.data['closing']
        L =[ [clos[n-j-1]-clos[i-j] for j in range(1, i+1)] for i in N] 
        return L
    
    # funtion that calculates the simple movign average :
    # funtion that calculates the simple movign average :
    def SMA(self, df, k):
        L = pd.DataFrame([])
        clos = df.data['closing']
        n = len(df.data)
        L = clos.rolling(k).mean()
        return L
    
 
    
    

    

### Application of the function :

In [44]:
ATL = Stock('ATL ASS')
k = 20
len(ATL.SMA(ATL, k)) # veify 

499

In [45]:
ATL.SMA(ATL, k)

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
494    106.1060
495    107.8560
496    109.5385
497    111.5685
498    113.1185
Name: closing, Length: 499, dtype: float64

### Task 8:

Let's extend the method that we implemented previously for visualization to include the two indicators you just implemented.

### Understand the algorithm and create the function:

- Let's understand the algorithm :

In [49]:
df = Stock('ATL ASS')
z = pd.DataFrame(df.data)
z = z [['date','closing','volume']]
   
z['date'] = pd.to_datetime(z['date'], format='%y%m%d') # 

# set the date as index
df1 = z.set_index('date') 

# Momentum parameters :
N = [5, 20, 120, 252]
M = df.moment(df, N)


# SMA parametrs
k = 5
sma = df.SMA(df,k)

# Add an ather axis to my plot
fig = make_subplots(rows=2,cols=1,
                shared_xaxes = True,vertical_spacing = 0.02,
                specs=[[{"secondary_y": True}], [{"secondary_y": False}]],
                )
#fig2 = make_subplots(specs=[[{"secondary_y": True}]]) 

# plot the closing price 
fig.add_trace(go.Scatter(x=df1.index,y=df1['closing'],name='Price'),secondary_y=False,) 

# plot the volume 
fig.add_trace(go.Bar(x=df1.index,y=df1['volume'],name='volume'),secondary_y=True,)

# plot the simple moving average
fig.add_trace(go.Scatter(x=df1.index[k:len(df.data)],y= sma ,name='Simple Moving average'),secondary_y=False,)

# plot the simple moving average
fig.add_trace(go.Bar(x=df1.index[N[1]:len(df.data)], y= M[1] ,name='Momentum'), row=2, col=1, secondary_y=False,)

# update the y axes
#fig2.update_yaxes(visible=True, secondary_y=True) 
#fig2.update_yaxes(visible=True, secondary_y=False)
fig.update_yaxes(title_text="Price", row=1, col=1,secondary_y=False)
fig.update_yaxes(title_text="Momentum", row=2, col=1)

#fig.update_yaxes(title_text="Momentum", row=3, col=1)
fig.add_hline(y=df.df_minp(),row=1, col=1,  annotation_text="min price" )
fig.add_hline(y=df.df_maxp(),row=1, col=1, line_color="green",  annotation_text="max price" )
# show the plots
fig.update_layout(legend=dict(orientation="h",))
fig.update_layout(height=700, width=1100,title={'text': 'ATL ASS', 'x': 0.4})
fig.show()
 
    
 
    
    


### The results :

In [50]:
class Stock:

    def __init__(self, name):
        self.name = name
        self.data = read_file(name+' .aspx')
       # z = pd.DataFrame(self.data)
       # self.df = z
        self.df = pd.DataFrame(self.data)

    # method that returns the maximum value in the Dataframe.
    def df_maxp(self):
        return max(self.data['closing'])

    # a method that returns the minimum value in the Dataframe.
    def df_minp(self):
        return min(self.data['closing'])
                
    # a method that returns the maximum volume in the Dataframe.
    def df_maxv(self):
        return max(self.data['volume'])
            
    # a method that returns the maximum quantity in the Dataframe.
    def df_maxq(self):
        return max(self.data['quantity'])

    #  create a method responsible for visualization. 
    def visualization(self,df, title):
   
        z = pd.DataFrame(df.data)
        z = z [['date','closing','volume']]
        
        z['date'] = pd.to_datetime(z['date'], format='%y%m%d') # 

        # set the date as index
        df1 = z.set_index('date') 

        # Momentum parameters :
        N = [5, 20, 120, 252]
        M = df.moment(df, N)


        # SMA parametrs
        k = 5
        sma = df.SMA(df,k)

        # Add an ather axis to my plot
        fig = make_subplots(rows=2,cols=1,
                        shared_xaxes = True,vertical_spacing = 0.02,
                        specs=[[{"secondary_y": True}], [{"secondary_y": False}]],
                        )
        #fig2 = make_subplots(specs=[[{"secondary_y": True}]]) 

        # plot the closing price 
        fig.add_trace(go.Scatter(x=df1.index,y=df1['closing'],name='Price of ' + title),secondary_y=False,) 

        # plot the volume 
        fig.add_trace(go.Bar(x=df1.index,y=df1['volume'],name='volume of ' + title),secondary_y=True,)

        # plot the simple moving average for 5 dats ago
        fig.add_trace(go.Scatter(x=df1.index[k:len(df.data)],y= sma ,name='Simple Moving average of ' + title),secondary_y=False,)
        
        # plot the momentum for 20 days ago
        fig.add_trace(go.Bar(x=df1.index[N[1]:len(df.data)], y= M[1] ,name='Momentum of ' + title), row=2, col=1, secondary_y=False,) 

        # update the y axes
        fig.update_yaxes(title_text="Price of " + title, row=1, col=1,secondary_y=False)
        fig.update_yaxes(title_text="Momentum of " + title, row=2, col=1)

        # ad lins for min and max in the data 
        fig.add_hline(y=df.df_minp(),row=1, col=1,  annotation_text="min price of " + title )
        fig.add_hline(y=df.df_maxp(),row=1, col=1, line_color="green",  annotation_text="max price of " + title )
        # show the plots
        fig.update_layout(legend=dict(orientation="h",))
        fig.update_layout(height=700, width=1100,title={'text': title + ' ASSURANCE', 'x': 0.5})
        fig.show()
        
    # method that calculates the momentum
    def moment(self, df, N):
        L = []
        n = len(df.data)
        clos = df.data['closing']
        L =[ [clos[n-j]-clos[n-i-j] for j in range(1, n-i)] for i in N]  
        return L
    
    # funtion that calculates the simple movign average :
    def SMA(self, df, k):
        L = pd.DataFrame([])
        clos = df.data['closing']
        n = len(df.data)
        #L = [sum(clos[n-k-i : n-i]/k) for i in range(1, n-k) ]
        L = clos.rolling(k).mean()
        return L
    
 
    
    

    

### ATLANTA ASSURANCE:

In [51]:
df = Stock('ATL ASS')
df.visualization(df, 'ATLANTA')

### AFMA ASSURANCE 

In [52]:
df = Stock('AFM AS')
df.visualization(df, 'AFMA')

### AGMA ASSURANCE 

In [53]:
df = Stock('AGM AS')
df.visualization(df, 'AGMA')

### SANLAM ASSURANCE

In [54]:
df = Stock('SAN ASS')
df.visualization(df, 'SANLAM')

### WAFA ASSURANCE 

In [55]:
df = Stock('WF ASS')
df.visualization(df, 'WAFA')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3d7c001b-4bb4-4e36-84d2-114c66908250' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>